<a href="https://colab.research.google.com/github/arjunprakash027/HandcraftedML/blob/main/Reinforcement_Learning/Basics/stable_baselines3_A2C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!apt install python-opengl
!apt install ffmpeg
!apt install xvfb
!pip3 install pyvirtualdisplay
!pip install stable-baselines3[extra]
!pip install gymnasium
!pip install huggingface_sb3
!pip install huggingface_hub
!pip install panda_gym

In [2]:
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

In [3]:
import os

import gymnasium as gym
import panda_gym

from huggingface_sb3 import load_from_hub, package_to_hub

from stable_baselines3 import A2C
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.env_util import make_vec_env

from huggingface_hub import notebook_login

In [9]:
env_id = "PandaReachDense-v3"

# Create the env
env = gym.make(env_id)

# Get the state space and action space
s_size = env.observation_space
a_size = env.action_space

print("_____OBSERVATION SPACE_____ \n")
print("The State Space is: ", s_size)
print("Sample observation", env.observation_space.sample())

print("\n _____ACTION SPACE_____ \n")
print("The Action Space is: ", a_size)
print("Action Space Sample", env.action_space.sample())

_____OBSERVATION SPACE_____ 

The State Space is:  Dict('achieved_goal': Box(-10.0, 10.0, (3,), float32), 'desired_goal': Box(-10.0, 10.0, (3,), float32), 'observation': Box(-10.0, 10.0, (6,), float32))
Sample observation {'achieved_goal': array([ 1.3185099,  8.949317 , -7.304697 ], dtype=float32), 'desired_goal': array([-7.4384775, -0.6147884,  4.741622 ], dtype=float32), 'observation': array([ 0.517758 , -2.9141364,  9.415086 , -3.4612422, -2.8228462,
       -0.7673421], dtype=float32)}

 _____ACTION SPACE_____ 

The Action Space is:  Box(-1.0, 1.0, (3,), float32)
Action Space Sample [0.99381214 0.968063   0.67743725]


In [10]:
env = make_vec_env(env_id=env_id, n_envs=4)
env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10.)

In [13]:
model = A2C(policy = "MultiInputPolicy",
            env = env,
            verbose=1)

Using cpu device


In [ ]:
model.learn(1_000_000)
model.save("a2c-PandaReachDense-v3")
env.save("vec_normalize.pkl")

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 45.5     |
|    ep_rew_mean        | -13.9    |
|    success_rate       | 0.143    |
| time/                 |          |
|    fps                | 275      |
|    iterations         | 100      |
|    time_elapsed       | 7        |
|    total_timesteps    | 2000     |
| train/                |          |
|    entropy_loss       | -4.26    |
|    explained_variance | 0.98     |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | -0.405   |
|    std                | 1        |
|    value_loss         | 0.0326   |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 45.4     |
|    ep_rew_mean        | -13.6    |
|    success_rate       | 0.14     |
| time/                 |          |
|    fps                | 282      |
|    iterations         | 200      |
|

In [ ]:
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize

eval_env = DummyVecEnv([lambda: gym.make("PandaReachDense-v3")])
eval_env = VecNormalize.load("vec_normalize.pkl", eval_env)
eval_env.render_mode = "rgb_array"
eval_env.training = False
eval_env.norm_reward = False

model = A2C.load("a2c-PandaReachDense-v3")
mean_reward, std_reward = evaluate_policy(model, eval_env)
print(f"Mean reward = {mean_reward:.2f} +/- {std_reward:.2f}")

In [12]:
notebook_login()
!git config --global credential.helper store

In [ ]:
from huggingface_sb3 import package_to_hub

package_to_hub(
    model=model,
    model_name=f"a2c-{env_id}",
    model_architecture="A2C",
    env_id=env_id,
    eval_env=eval_env,
    repo_id=f"Arjunrao/a2c-{env_id}",
    commit_message="Initial commit",
)